0. Se hace la instalación de los paquetes SIG

In [1]:
!pip install wheel
!pip install rasterio
!pip install rarfile
#!pip install richdem


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


1. Se procede con la carga de las librerias de trabajo

In [2]:
#Se cargan las librerias SIG
import rasterio
#import richdem as rd
from PIL import Image

#Para cargar los archivos automáticamente del drive
from google.colab import drive
drive.mount('/content/drive')

#Se procede con la carga de las librerias
import pandas as pd
import numpy as np
from pyproj import Transformer

#Se procede con la carga de las librerias de descompresión
import rarfile
import zipfile
import os

Mounted at /content/drive


**Funciones de Trabajo**
1. Se hace el procesamiento espacial

In [ ]:
def rejilla_indice(ruta_imagen,ruta_imagen_2):

  #Se procede con la carga de los archivos
  # Abrir la imagen
  imagen = Image.open(ruta_imagen_2)

  # Convertir a arreglo para acceder a los píxeles
  pixels = imagen.load()

  xm=[]; ym=[]; latm=[]; lonm=[]; colm=[]; rowm=[]; NDVI=[]

  # Abrir la imagen con Rasterio
  with rasterio.open(ruta_imagen) as src:
      banda1 = src.read(1)  #Aquí se obtiene el valor del indice de vegetación
      crs = src.crs
      print("CRS de la imagen:", crs)
      # Obtener las coordenadas del píxel (0, 0)
      row, col = 0, 0
      #row, col = src.index(col, row)  # Invierte fila y columna si es necesario
      x, y = src.xy(col, row)
      #altitud=dem.get_z(x,y)
      #print("Altitud del píxel (0, 0):", altitud)
      print("Coordenadas del píxel (0, 0):", x, y)

      # Obtener las coordenadas de todos los píxeles
      m1=0

      print("el ancho es",src.width)
      print("el alto es",src.height)

      for row in range(src.height):
          for col in range(src.width):

              # Obtener el valor del píxel
              x, y = src.xy(row, col)
              valor_pixel = banda1[row, col]

              if valor_pixel != -9999.0:

                  # Hacer algo con las coordenadas, por ejemplo, guardarlas en un archivo
                  m1+=1
                  #print(m1)
                  #print(f"Píxel ({col}, {row}): {x}, {y}")

                  valor_pixel_2 = pixels[row, col]  # Nota: PIL usa (columna, fila)

                  # Normalizar cada valor del pixel (R, G, B, A)
                  valor_pixel_2 = tuple(val / 255 for val in valor_pixel_2)

                  transformer = Transformer.from_crs(crs, "EPSG:4326")
                  lon, lat = transformer.transform(x, y)
                  #print(f"Píxel ({col}, {row}): {lon}, {lat}")
                  #print("Valor del pixel:", valor_pixel)
                  #print("Valor del pixel R-G-B-NIR:", valor_pixel_2)
                  xm.append(x); ym.append(y)
                  latm.append(lat); lonm.append(lon)
                  colm.append(col); rowm.append(row)
                  NDVI.append(valor_pixel)

      df=pd.DataFrame(np.column_stack((xm,ym, latm, lonm, colm, rowm,NDVI)))
      df.columns = ['UTM-x', 'UTM-y', 'longitud', 'latitud','col','row','NDVI']

      return df

2. Se hace la estandarización del análisis espacial


*   Resolucion Específica.

In [4]:
def IDW_Index(df):

        xm=np.array(df['longitud'])
        ym=np.array(df['latitud'])
        zms=np.array(df['NDVI'])

        xm_min=df['longitud'].min();ym_min=df['latitud'].min()
        xm_max=df['longitud'].max();ym_max=df['latitud'].max()

        #2. Se procede con la creación de la rejilla
        res=5
        xg=np.linspace(xm_min, xm_max, res)
        yg=np.linspace(ym_min, ym_max, res)

        #0. Parametros
        perc=95
        zidw=np.zeros((res,res));zidwd=np.zeros((res,res))

        #1. Se procede con la construcción del modelo IDW-KNN (Key Nearest)
        nKNN=10; dknn=np.zeros((nKNN,1)); dknnd=np.zeros((nKNN,1))
        ds=np.zeros((nKNN,1))

        idm=np.zeros((res*res,1));xm_2=np.zeros((res*res,1));ym_2=np.zeros((res*res,1));zms_2=np.zeros((res*res,1))

        m4=-1
        for i in range(res):
          for j in range(res):
            #i=3;j=3
            m4+=1
            idm[m4] = m4  # Asigna el índice a idm[m4]
            xm_2[m4] = xg[i]  # Asigna el valor correspondiente de xg[i] a xm_2[m4]
            ym_2[m4] = yg[j]  # Asigna el valor correspondiente de yg[j] a ym_2[m4]
            dx=(xg[i,]-xm)**2;dy=(yg[j,]-ym)**2
            d=np.sqrt(dx+dy);ds=np.array(sorted(d))[0:nKNN,]

            for k in range(nKNN):
              dknn[k,]=np.where(d==ds[k,])[0][0]
              dknnd[k,]=np.sqrt((xg[i,]-xm[int(dknn[k,].item()),])**2+(yg[j,]-ym[int(dknn[k,].item()),])**2)
              zidwd[i,j]=zidwd[i,j]+((1/dknnd[k,0])**3)

              #Las proyecciones espaciales del centro de coordenadas de las imagenes disponibles
              #Sobre unos puntos cualquiera xg,yg
              zidw[i,j]+=((1/dknnd[k,0])**3)*zms[int(dknn[k,].item()),]   #Terreno

            zidw[i,j]=zidw[i,j]/zidwd[i,j]
            zms_2[m4,]=zidw[i,j]

            #'''''''''''''''''''''''''''''''''''''''''''''''''
            #2. Se procede con el suavizamiento del Terreno
            #'''''''''''''''''''''''''''''''''''''''''''''''''
            zidws=np.zeros((res+1,res+1))
            zidws[0,1:res+1]=xg.flatten()
            zidws[1:res+1,0]=yg.flatten()
            zidws[1:res+1,1:res+1]=zidw

            dfidw=pd.DataFrame(zidws)
            dfidw_2=pd.DataFrame(np.column_stack((idm,xm_2,ym_2, zms_2)))
            dfidw_2.columns = ['id','long-xm', 'long-ym', 'NDVI']

        return dfidw, dfidw_2

3. Se procede con la caracterización del riesgo

In [ ]:
def Riesgo(dfm,XC):

        nXC=np.zeros((25,1))
        XLDA=np.array(df_2[1]['NDVI'])

        for k1 in range(2):
           XC=np.sort(XC)[::-1]
           for k in range(25):

               ncj=np.where(np.min((XC-XLDA[k,])**2)==(XC-XLDA[k,])**2)[0][0]
               XC[ncj,]=(XC[ncj,]+XLDA[k,])/2
               nXC[k,]=ncj

        dfm['Riesgo']=nXC+1

        return dfm,XC

## ** Ejecución del Análisis Espacial**
1. Se hace el proceso de descompresión de las imagenes

In [9]:
#Se procede con el ingreso de la información
indice=input("Ingresar el nombre del indice de vegetación:")
año=input("Ingresar el año de análisis:")

#rutas de las carpetas
ruta_carpeta='/content/drive/MyDrive/Software-EAFIT-DMU/Software_Puerta/'+indice+'_'+año+'/'
ruta_destino=ruta_carpeta

os.makedirs(ruta_destino, exist_ok=True)

for archivo in os.listdir(ruta_carpeta):
    print(archivo)
    if archivo.endswith('.zip'):  # Verificar que sea un archivo .zip
        ruta_zip = os.path.join(ruta_carpeta, archivo)
        print(f"Procesando: {ruta_zip}")

       # Abrir y extraer el archivo .zip
        with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
            # Extraer directamente en la carpeta de destino
            zip_ref.extractall(ruta_destino)
            print(f"Archivos extraídos en: {ruta_destino}")
            print(zip_ref.namelist())

            if archivo[0:5]+zip_ref.namelist()[0] in os.listdir(ruta_carpeta):
              print(f"El archivo '{zip_ref.namelist()[0]}' está en la carpeta.")
              os.remove(ruta_carpeta+zip_ref.namelist()[0])
              os.remove(ruta_carpeta+zip_ref.namelist()[1])
            else:
              print(f"El archivo '{zip_ref.namelist()[0]}' No está en la carpeta.")
              os.rename(ruta_carpeta+zip_ref.namelist()[0],ruta_carpeta+archivo[0:5]+zip_ref.namelist()[0])
              os.rename(ruta_carpeta+zip_ref.namelist()[1],ruta_carpeta+archivo[0:5]+zip_ref.namelist()[1])


Ingresar el nombre del indice de vegetación:NDVI
Ingresar el año de análisis:2024
049. perimetro__prev_NDVI_06dic2024.zip
Procesando: /content/drive/MyDrive/Software-EAFIT-DMU/Software_Puerta/NDVI_2024/049. perimetro__prev_NDVI_06dic2024.zip
Archivos extraídos en: /content/drive/MyDrive/Software-EAFIT-DMU/Software_Puerta/NDVI_2024/
['perimetro__prev_NDVI_ColorMap_06dic2024.tiff', 'perimetro__prev_NDVI_06dic2024.tiff']
El archivo 'perimetro__prev_NDVI_ColorMap_06dic2024.tiff' está en la carpeta.
048. perimetro__prev_NDVI_01dic2024.zip
Procesando: /content/drive/MyDrive/Software-EAFIT-DMU/Software_Puerta/NDVI_2024/048. perimetro__prev_NDVI_01dic2024.zip
Archivos extraídos en: /content/drive/MyDrive/Software-EAFIT-DMU/Software_Puerta/NDVI_2024/
['perimetro__prev_NDVI_ColorMap_01dic2024.tiff', 'perimetro__prev_NDVI_01dic2024.tiff']
El archivo 'perimetro__prev_NDVI_ColorMap_01dic2024.tiff' está en la carpeta.
047. perimetro__prev_NDVI_06nov2024.zip
Procesando: /content/drive/MyDrive/Softwar

2. Se procede con el procesamiento de las imagenes

In [10]:
#Se procede con el ingreso de la información
indice=input("Ingresar el nombre del indice de vegetación:")
año=input("Ingresar el año de análisis:")

# Ruta de la carpeta que contiene los archivos
nxl='/content/drive/MyDrive/Software-EAFIT-DMU/Software_Puerta/'+indice+'_'+año+'/'
nombre='ColorMap'
contador=0

#Se procede con la creación de las semillas del riesgo
XC=np.random.uniform(0,1,5)
XC=np.sort(XC)
print(XC)

# Iterar por los archivos en la carpeta
# Aquí identificamos solo los que tienen los valores del Indice NDVI
for archivo in os.listdir(nxl):
    if nombre in archivo:
        contador += 1
        print(archivo)

print("El número de archivos que posee la carpeta es:", contador)

# Se procede con la identificación de cada uno de los archivos
for k in range(1,contador+1):
  print(k)

  for archivo in os.listdir(nxl):
    if nombre in archivo and str(k).zfill(3) in archivo[0:4]:  # Verificar si el nombre contiene la palabra específica
        print(k)
        archivo_2=archivo
        archivo=archivo_2.replace(str('_')+nombre,'')
        print(archivo)
        print(archivo_2)

        #dem = rd.LoadGDAL(nxl+archivo_2,no_data=-9999)

        # Abrir el archivo raster con rasterio
        with rasterio.open(nxl + archivo_2) as src:
          # Leer la primera banda de datos como un arreglo NumPy
          dem = src.read(1)

          # Obtener el valor de "no_data" del archivo, si está definido
          no_data_value = src.nodata if src.nodata is not None else -9999

          # Reemplazar los valores "no_data" por NaN para manejarlo mejor
          dem = np.where(dem == no_data_value, np.nan, dem)

        # Abrir la imagen con rasterio
        with rasterio.open(nxl+archivo_2) as dataset:
              # Obtener los metadatos
              metadatos = dataset.meta
              print(metadatos)

              # Acceder a un metadato específico
              print(metadatos['crs'])  # Imprime el sistema de coordenadas
              print(metadatos['transform'])  # Imprime la matriz de transformación

        df=rejilla_indice(nxl+archivo,nxl+archivo_2)
        #print(df)

        if k==1:
          # Ruta del archivo Excel
            ruta_excel = nxl+'Informe '+str(indice)+' Espacial '+str(nxl[47:51])+'.xlsx'

            with pd.ExcelWriter(ruta_excel, engine='openpyxl') as writer:
              df.to_excel(writer, index=False, sheet_name=str(archivo_2[35:44]))

            ruta_excel_2 = nxl+'Informe '+str(indice)+' IDW '+str(nxl[47:51])+'.xlsx'
            df_2=IDW_Index(df)

            with pd.ExcelWriter(ruta_excel_2, engine='openpyxl') as writer:
              df_2[0].to_excel(writer, index=False, sheet_name=str(archivo_2[35:44]))

            ruta_excel_3 = nxl+'Informe '+str(indice)+' QGIS '+str(nxl[47:51])+'.xlsx'
            df2x=Riesgo(df_2[1],XC)

            with pd.ExcelWriter(ruta_excel_3, engine='openpyxl') as writer:
              df2x[0].to_excel(writer, index=False, sheet_name=str(archivo_2[35:44]))

            XC=df2x[1]
            print("Los clusters son:",XC)

        else:

            # Ruta del archivo Excel
            ruta_excel = nxl+'Informe '+str(indice)+' Espacial '+str(nxl[47:51])+'.xlsx'

            with pd.ExcelWriter(ruta_excel, mode='a', engine='openpyxl') as writer:
              df.to_excel(writer, index=False, sheet_name=str(archivo_2[35:44]))

            df_2=IDW_Index(df)

            ruta_excel_2 = nxl+'Informe '+str(indice)+' IDW '+str(nxl[47:51])+'.xlsx'
            with pd.ExcelWriter(ruta_excel_2, mode='a', engine='openpyxl') as writer:
              df_2[0].to_excel(writer, index=False, sheet_name=str(archivo_2[35:44]))

            ruta_excel_3 = nxl+'Informe '+str(indice)+' QGIS '+str(nxl[47:51])+'.xlsx'
            df2x=Riesgo(df_2[1],XC)

            with pd.ExcelWriter(ruta_excel_3, mode='a', engine='openpyxl') as writer:
              df2x[0].to_excel(writer, index=False, sheet_name=str(archivo_2[35:44]))

            XC=df2x[1]
            print("Los clusters son:",XC)

Ingresar el nombre del indice de vegetación:NDVI
Ingresar el año de análisis:2024
[0.02718168 0.04150356 0.57035843 0.78112429 0.78768016]
049. perimetro__prev_NDVI_ColorMap_06dic2024.tiff
048. perimetro__prev_NDVI_ColorMap_01dic2024.tiff
047. perimetro__prev_NDVI_ColorMap_06nov2024.tiff
046. perimetro__prev_NDVI_ColorMap_07oct2024.tiff
045. perimetro__prev_NDVI_ColorMap_27sep2024.tiff
044. perimetro__prev_NDVI_ColorMap_17sep2024.tiff
043. perimetro__prev_NDVI_ColorMap_07sep2024.tiff
042. perimetro__prev_NDVI_ColorMap_28ago2024.tiff
041. perimetro__prev_NDVI_ColorMap_18ago2024.tiff
040. perimetro__prev_NDVI_ColorMap_13ago2024.tiff
039. perimetro__prev_NDVI_ColorMap_29jul2024.tiff
038. perimetro__prev_NDVI_ColorMap_24jul2024.tiff
037. perimetro__prev_NDVI_ColorMap_14jul2024.tiff
036. perimetro__prev_NDVI_ColorMap_09jul2024.tiff
035. perimetro__prev_NDVI_ColorMap_30abr2024.tiff
034. perimetro__prev_NDVI_ColorMap_15abr2024.tiff
033. perimetro__prev_NDVI_ColorMap_10abr2024.tiff
032. perime

3. Se procede con la configuración del archivo Clima
  *   Aquí la fecha más antigua es el primer valor que debe aparecer.

In [14]:
#Se procede con el ingreso de la información
indice=input("Ingresar el nombre del indice de vegetación:")
año=input("Ingresar el año de análisis:")

# Ruta de la carpeta que contiene los archivos
nxl='/content/drive/MyDrive/Software-EAFIT-DMU/Software_Puerta/'+indice+'_'+año+'/Clima_'+indice+'_'+año+'.xlsx'
print(nxl)

# Leer los datos de la hoja
dfc = pd.read_excel(nxl, sheet_name=0)

# Invertir el orden de las filas
df_invertido = dfc.iloc[::-1]

ruta_salida = 'Clima_'+indice+'_'+año+'_O.xlsx'  # Cambia esto por la ruta de salida
nxl2='/content/drive/MyDrive/Software-EAFIT-DMU/Software_Puerta/'+indice+'_'+año+'/'+ruta_salida
df_invertido.to_excel(nxl2, index=False, sheet_name='Sheet 1')

print(f"El archivo con las filas invertidas se guardó en: {nxl2}")
df_invertido.to_excel(ruta_salida, index=False, sheet_name='Sheet 1')

print(f"El archivo con las filas invertidas se guardó en: {ruta_salida}")

Ingresar el nombre del indice de vegetación:NDVI
Ingresar el año de análisis:2024
/content/drive/MyDrive/Software-EAFIT-DMU/Software_Puerta/NDVI_2024/Clima_NDVI_2024.xlsx
El archivo con las filas invertidas se guardó en: /content/drive/MyDrive/Software-EAFIT-DMU/Software_Puerta/NDVI_2024/Clima_NDVI_2024_O.xlsx
El archivo con las filas invertidas se guardó en: Clima_NDVI_2024_O.xlsx
